# How to downloade data from ENTSO-e's Transparency Platform

The **European Network of Transmission System Operators for Electricity** (ENTSO-e) is the organization for national Transmission System Operators (TSOs). It represents 39 European TSOs and trangresses the EU borders. 

ENTSO-e provide something known as the [**Transparency Platform**](https://transparency.entsoe.eu/), which is a large data bank for the European electricity system. Their API service is often very handy when working with numerical models of the electricity system. 

In this notebook you will learn how to download data from the Transparency Platform's Api service using a python package called **entsoe-py**. You can have a look at its [**documentation page**](https://github.com/EnergieID/entsoe-py) to see how to use the package. 

In order to run the code in this notebook, you will need to complete three steps beforehand:
1. Install **entsoe-py** in you conda environment.
2. Acquire an API key for the Transparency Platform.
3. Save the API key as en _environment variable_ in you conda environment.

We go through these three steps below in [**Part 0**](#Part-0:-Required-steps-to-run-the-code). 

In [**Part 1**](#Part-1:-Code-chunks-for-downloading-data), we will be downloading three variables from the Transparency Platform that are important model inputs in many electricity system models. These are

- **Forecasted load**: We use the forecasted load on the day-ahead market to specify how demand exogenously varies on the hourly frequency.
- **Forecasted intermittent supply**: We use the forecasted intermittent generation on the day-ahead market to specify how intermittent electricity generation exogenously varies on the hourly frequency.
- **Spot prices**: We can use the spot prices in connected electricity areas to invoke a 'small open economy'-assumption. Essentially, we will assume that international prices are constant and that trade with the domestic country does not affect affect foreign prices.

## Part 0: Required steps to run the code

___Step 1: Install **entsoe-py**___

This done quite easily. First, open the anaconda prompt (for Windows) or terminal (for Mac/Linux) and activate your conda environment:

> `conda activate insert_your_environment_name`

Next, install the package:

> `python -m pip install entsoe-py`

Note, my installed python version 3 is just called python. Yours might be called python3 if you are a little more tidy than me.

___Step 2: Acquire an API key___

ENTSO-e provides a free API key for registered users of the Platform. You just need to create an account. Then follow [**these guidelines for getting an API key**](https://transparency.entsoe.eu/content/static_content/download?path=/Static%20content/API-Token-Management.pdf) under the headline "_If I lose my Token or I don’t have any, how can I generate a new one?_"

__Step 3: Save the API key in your environment__

Saving API key as an environment variable on your local computer implies the API key is not publicly accessible and only activate whenever you conda environment is activated. When can then us the python-internal **os**-package to acquire the API key from your local laptop. But when you share the notebook, the notebook will never share your API key.

There some great guides for doing this. For Windows, you can follow [**Hide Your Secret Keys With Conda**](https://medium.com/analytics-vidhya/hide-your-secret-keys-with-conda-1d22e42b82ed). For MAC and Linux users you can follow [**How to set environment variables in a conda virtual environment**](https://guillaume-martin.github.io/saving-environment-variables-in-conda.html). Below, I show it for Windows users.

Open you the anaconda prompt and activate your environment if you haven't done so:

> `conda activate insert_your_environment_name` 

Now navigate to the folder, where you environment is located by entering

> `cd %CONDA_PREFIX%`

Create the two folders **activate.d** and **deactivate.d**:

> `mkdir .\etc\conda\activate.d` <br>
> `mkdir .\etc\conda\deactivate.d`

Finally, create a batch-file in the two folders:

> `type NUL > .\etc\conda\activate.d\envv-awscreds.bat`<br>
> `type NUL > .\etc\conda\deactivate.d\envv-awscreds.bat`

You should now navigate to the folder where the two batch files are stored and edit the two batch files in a text editor (e.g. notepad). The directory will be visible from the terminal. For instance, my terminal reads 

> <span style='background:black'> <span style='color:white'> (EEotGT23) C:\Users\xnh825\Anaconda3\envs\EEotGT23>  </span>  </span>

where __EEotGT23__ is the environment name. The directories therefore read

    C:\Users\xnh825\Anaconda3\envs\EEotGT23\etc\conda\activate.d\
    C:\Users\xnh825\Anaconda3\envs\EEotGT23\etc\conda\deactivate.d\
    
In __activate.d\envv-awscreds.bat__ you write

> `set ENTSOE_API_KEY=Insert_your_api_key`

In __deactivate.d\envv-awscreds.bat__ you write

> `set ENTSOE_API_KEY=`

From now on, every time you activate your environment, the environment variable __ENTSOE_API_KEY__ will be available in your local computer. 

We are finally ready to run the code below.

## Part 1: Code-chunks for downloading data

We start by importing a few packages:

In [1]:
import pandas as pd, numpy as np, os, pickle
from entsoe import EntsoePandasClient, Area

Let's specify an output folder:

In [2]:
direc = os.getcwd()
data_dir = os.path.join(direc,'CleanedData')

Choose the year you want to collect data for (currently only one year is supported, but you can easily adopt the notebook to collect for multiple years):

In [3]:
year = 2022

Choose the area of interest:

In [4]:
g_area = ['DK_1','DK_2']

Specify the international markets connected to the domestic market (you can use [**NordPool's map**](https://www.nordpoolgroup.com/en/Market-data1/#/nordic/map)):

In [5]:
export_g_areas = ['DE_LU','NL','SE_3','NO_2','SE_4'] 
import_g_areas = ['DE_LU','NL','NO_2','SE_4']
connected_g_areas = np.unique(export_g_areas + import_g_areas)

We now retrieve you locally stored API key. Importantly, the environment variable should be named __ENTSOE_API_KEY__:

In [6]:
entsoe_api_key = os.environ.get("ENTSOE_API_KEY").replace("'","")

Using the API key, we can activate the client:

In [7]:
client = EntsoePandasClient(api_key=entsoe_api_key)

Given the chosen year, create choose the first and last hour of that year:

In [13]:
start_str, end_str = str(year)+'-01-01T00',str(year)+'-12-31T23', 
start_CET = pd.Timestamp(start_str,tz='Europe/Copenhagen'); start_UTC = start_CET.tz_convert('UTC')
end_CET = pd.Timestamp(end_str,tz='Europe/Copenhagen'); end_UTC = end_CET.tz_convert('UTC')
h_index = pd.Index(pd.date_range(start_CET,end_CET,freq='H'),name='HourCET/CEST')

### 1.1 Download load

Download the data using the client:

In [14]:
df_demand = pd.concat([
    pd.concat([client.query_load_forecast(Area[x], start=start_CET,end=end_CET, process_type='A01').rename_axis('HourCET/CEST').reindex(h_index).assign(g_E = x).set_index('g_E',append=True)['Forecasted Load'].rename('ForecastedLoad_MWh') for x in g_area],axis=0),
    pd.concat([client.query_load(Area[x], start=start_CET,end=end_CET).rename_axis('HourCET/CEST').reindex(h_index).assign(g_E=x).set_index('g_E',append=True)['Actual Load'].rename('ActualLoad_MWh') for x in g_area],axis=0)
],axis=1).reset_index()

Add some helpfull variables:

In [16]:
df_demand['HourUTC'] = df_demand['HourCET/CEST'].dt.tz_convert('UTC')
df_demand['HourOfTheDay'] = df_demand['HourCET/CEST'].dt.hour
df_demand['Weekday'] = df_demand['HourCET/CEST'].dt.weekday 
df_demand['Week'] = df_demand['HourCET/CEST'].dt.isocalendar().week
df_demand['Month'] = df_demand['HourCET/CEST'].dt.month
df_demand['h'] = 1
df_demand['h'] = df_demand.groupby('g_E')['h'].cumsum()
df_demand['g_E'] = df_demand['g_E'].str.replace('_','')
new_col_order = ['g_E','HourUTC','HourCET/CEST','Month','Week','Weekday','HourOfTheDay','h','ForecastedLoad_MWh','ActualLoad_MWh']
df_demand = df_demand[new_col_order].sort_values(new_col_order).reset_index(drop=True)

Save the data as a pickle:

In [17]:
file_path = os.path.join(data_dir,'Load_'+''.join(df_demand['g_E'].unique().tolist())+'_'+str(year))
df_demand.to_pickle(file_path)

### 1.2 Download intermittent generation

Download forecasts using the client:

In [18]:
d_cols = {'Wind Offshore':'WindOffshore','Wind Onshore':'WindOnshore','Hydro Run-of-river and poundage':'ROR'}
forecasts = pd.concat([client.query_wind_and_solar_forecast(Area[x], start=start_CET, end=end_CET, process_type='A01').rename_axis('HourCET/CEST').reindex(h_index).reset_index().assign(g_E=x).rename(columns=d_cols) for x in g_area],axis=0).melt(id_vars=['HourCET/CEST','g_E'], var_name='hvt',value_name='Forecast_MWh')

Download actual generation using the client:

In [19]:
psr_types = {
    'B11':'Hydro Run-of-river and poundage',
    'B16':'Solar',
    'B18':'Wind Offshore',
    'B19':'Wind Onshore'
}
t_combs = [x for x in pd.MultiIndex.from_product([g_area,psr_types.keys()],names=['g_E','psr']) if x!=('DK_2','B11')]
actual =  pd.concat([client.query_generation(Area[x[0]], start=start_CET, end=end_CET, psr_type=x[1]).rename_axis('HourCET/CEST').reindex(h_index).assign(g_E=x[0],hvt=psr_types[x[1]]).set_index(['g_E','hvt'],append=True)[psr_types[x[1]]].rename('Actual_MWh') for x in t_combs],axis=0).reset_index().replace({'hvt':d_cols})

Merge forecasts and actual generation:

In [20]:
df_supply = forecasts.merge(actual, how='outer')

Add some helpfull variables:

In [21]:
df_supply['HourUTC'] = df_supply['HourCET/CEST'].dt.tz_convert('UTC')
df_supply['HourOfTheDay'] = df_supply['HourCET/CEST'].dt.hour
df_supply['Weekday'] = df_supply['HourCET/CEST'].dt.weekday 
df_supply['Week'] = df_supply['HourCET/CEST'].dt.isocalendar().week
df_supply['Month'] = df_supply['HourCET/CEST'].dt.month
df_supply['h'] = 1
df_supply['h'] = df_supply.groupby(['g_E','hvt'])['h'].cumsum()
df_supply['g_E'] = df_supply['g_E'].str.replace('_','')
new_col_order = ['g_E','hvt','HourUTC','HourCET/CEST','Month','Week','Weekday','HourOfTheDay','h','Forecast_MWh','Actual_MWh']
df_supply = df_supply[new_col_order].sort_values(new_col_order).reset_index(drop=True)

Save the data as a pickle:

In [22]:
file_path = os.path.join(data_dir,'IntermittentSupply_'+''.join(df_supply['g_E'].unique().tolist())+'_'+str(year))
df_supply.to_pickle(file_path)

### 1.3 Download spot market prices

In [23]:
df_prices = pd.concat([client.query_day_ahead_prices(Area[g], start=start_CET, end=end_CET).reindex(h_index).rename('SpotPrice_€/MWh').to_frame().rename_axis('HourCET/CEST').assign(g_E=g).set_index('g_E',append=True)['SpotPrice_€/MWh'] for g in list(connected_g_areas)+g_area],axis=0).reset_index()

In [24]:
df_prices['HourUTC'] = df_prices['HourCET/CEST'].dt.tz_convert('UTC')
df_prices['HourOfTheDay'] = df_prices['HourCET/CEST'].dt.hour
df_prices['Weekday'] = df_prices['HourCET/CEST'].dt.weekday 
df_prices['Week'] = df_prices['HourCET/CEST'].dt.isocalendar().week
df_prices['Month'] = df_prices['HourCET/CEST'].dt.month
df_prices['h'] = 1
df_prices['h'] = df_prices.groupby('g_E')['h'].cumsum()
df_prices['g_E'] = df_prices['g_E'].str.replace('_','')
new_col_order = ['g_E','HourUTC','HourCET/CEST','Month','Week','Weekday','HourOfTheDay','h','SpotPrice_€/MWh']
df_prices = df_prices[new_col_order].sort_values(['g_E','h']).reset_index(drop=True)

In [25]:
file_path = os.path.join(data_dir,'SpotPrices_'+''.join(df_prices['g_E'].unique().tolist())+'_'+str(year))
df_prices.to_pickle(file_path)

### 1.4 Download transmission capacities

In [ ]:
# start_str, end_str = str(2022)+'-01-01T00',str(2022)+'-12-31T23', 
# start_CET = pd.Timestamp(start_str,tz='Europe/Copenhagen'); start_UTC = start_CET.tz_convert('UTC')
# end_CET = pd.Timestamp(end_str,tz='Europe/Copenhagen'); end_UTC = end_CET.tz_convert('UTC')
# df_demand = client.query_load(Area[g_area], start=start_CET,end=end_CET).reset_index().rename(columns={'index':'HourCET/CEST','Actual Load':'Load_MWh'})

In [ ]:
# from scipy import signal
# from matplotlib import pyplot as plt

In [ ]:
# y = df_demand['Load_MWh']
# # y = (y-y.mean())/y.std()
# seconds_pr_hour = 60*60
# sampling_frequency = 1/seconds_pr_hour
# n = len(y)
# fstep = sampling_frequency / n
# freq, Pxx = signal.periodogram(y, fs=sampling_frequency)
# # Pxx = np.fft.rfft(y)
# # freq = np.arange(len(Pxx)) * fstep
# x_scale = pd.Series(freq/fstep)
# plt.semilogy(np.log(x_scale[2:]),Pxx[2:]*seconds_pr_hour*2)
# # fig, ax = plt.subplots()
# # ax.plot(x_scale[1:],np.log(Pxx[1:]))
# # ax.vlines(ymin=Pxx[1:].min(), ymax=Pxx[23].max(), x=np.log(24),color='black')

In [ ]:
# x = df_demand.groupby('HourOfTheDay')['Load_MWh'].agg(['mean','std'])
# x['upper'] = x['mean']+1.96*x['std']
# x['lower'] = x['mean']-1.96*x['std']
# x.drop(columns='std').plot()